# 커리큘럼 preview quiz 검수

In [1]:
import pandas as pd
import re
import json
import random
import csv
from tqdm.notebook import tqdm
from collections import Counter

In [20]:
df_mid = pd.read_csv("DB_MID.csv")
df_elem = pd.read_csv("DB_Elem.csv")
df_high =pd.read_csv("DB_High.csv")

In [21]:
df_mid.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'title', 'new_DES',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 30', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34'],
      dtype='object')

In [101]:
def preview_quiz_check_valid(data:pd.DataFrame):
    # 에러 유무 확인
    error_found = False
    
    # 변수 가져오기
    for i in tqdm(range(len(data)), total = len(data)):
        # quiz 1
        quiz_1 = data["Quiz 1"][i]
        korean_1 = data["Korean 1"][i]
        option_1_1 = data["Option 1-1"][i]
        option_1_2 = data["Option 1-2"][i]
        option_1_3 = data["Option 1-3"][i]
        answer_1 = data["Answer 1"][i]
        index_1 = data["Index 1"][i]

        # quiz 2
        quiz_2 = data["Quiz 2"][i]
        korean_2 = data["Korean 2"][i]
        option_2_1 = data["Option 2-1"][i]
        option_2_2 = data["Option 2-2"][i]
        option_2_3 = data["Option 2-3"][i]
        answer_2 = data["Answer 2"][i]
        index_2 = data["Index 2"][i]

        # 샘플스크립트
        sample_script_kor = data.loc[i,"샘플 스크립트 (한국어)"]

        # QUIZ 1 확인
        # 필드값 누락 체크
        if (    not quiz_1
                or not korean_1
                or not option_1_1
                or not option_1_2
                or not option_1_3
                or not answer_1
                or not index_1
            ):
            print(f"\nQuiz 1 {i+2}번째 행 : 필드값 누락 체크")
            error_found = True

        # 1. 빈칸이 하나 이상있고, 두 개 이상이 없는지 체크
        appropriate_blank = r"_+"
        underscores = re.findall(appropriate_blank, quiz_1)

        if len(underscores) > 1:
            print(f"\nQuiz 1 {i+2}번째 행 : Blank error - 밑줄이 두 개 이상 존재")
            error_found = True
        elif len(underscores) ==1:
            pass
        else:
            print(f"\nQuiz 1 {i+2}번째 행 : Blank error - 밑줄이 존재하지 않음")
            error_found = True

        # 1-1 한국어 해석에 빈칸이 있는지
        appropriate_blank_korean = r'[_~]+'
        korean_underscores = re.findall(appropriate_blank_korean, korean_1)
        if len(korean_underscores) >= 1:
            print(f"\nQuiz 1 {i+2}번째 행 : 한국어 해석에 밑줄 있음")
            error_found = True

        # 2. 정답이 옵션에 포함되어 있는지 체크
        options_1 = [option_1_1, option_1_2, option_1_3]

        # answer가 options에 있는지 확인
        if answer_1 not in options_1:
            print(f"\nQuiz 1 {i+2}번째 행 : Answer not in options error")
            error_found = True

        # answer가 숫자로 이루어져있는지 확인
        if isinstance(index_1, str) and not index_1.isnumeric():
            print(f"\nQuiz 1 {i+2}번째 행 : Answer in options error")
            error_found = True

        # 3. 정답의 인덱스가 answer_index와 일치하는지 확인
        index_1 = int(index_1)
        if options_1[index_1 - 1] != answer_1:
            print((f"\nQuiz 1 {i+2}번째 행 : 정답의 인덱스가 answer_index와 불일치"))
            #print(f"{options_1}의 정답이 {answer_1}이지만 index {index_1}로 적혀있는 에러")
            error_found = True

        # 4. 문장에 두 대화문이 있는지 확인
        appropriate_blank_english = r'A:|B:'
        english_underscores = re.findall(appropriate_blank_english, quiz_1)
        if len(english_underscores) >= 4:
            print(f"\nQuiz 1 {i+2}번째 행 : 퀴즈에 대화문이 2개 있음")
            error_found = True

        # QUIZ 2 확인
        # 필드값 누락 체크
        if (    not quiz_2
                or not korean_2
                or not option_2_1
                or not option_2_2
                or not option_2_3
                or not answer_2
                or not index_2
            ):
            print(f"\nQuiz 2 {i+2}번째 행 : 필드값 누락 체크")
            error_found = True

        # 1. 빈칸이 하나 이상있고, 두 개 이상이 없는지 체크
        appropriate_blank = r"_+"
        underscores = re.findall(appropriate_blank, quiz_2)

        if len(underscores) > 1:
            print(f"\nQuiz 2 {i+2}번째 행 : Blank error - 밑줄이 두 개 이상 존재")
            error_found = True
        elif len(underscores) ==1:
            pass
        else:
            print(f"\nQuiz 2 {i+2}번째 행 : Blank error - 밑줄이 존재하지 않음")
            error_found = True
        
        # 1-1 한국어 해석에 빈칸이 있는지
        appropriate_blank_korean = r'[_~]+'
        korean_underscores = re.findall(appropriate_blank_korean, korean_2)
        if len(korean_underscores) >= 1:
            print(f"\nQuiz 2 {i+2}번째 행 : 한국어 해석에 밑줄 있음")
            error_found = True

        # 2. 정답이 옵션에 포함되어 있는지 체크
        options_2 = [option_2_1, option_2_2, option_2_3]

        # answer가 options에 있는지 확인
        if answer_2 not in options_2:
            print(f"\nQuiz 2 {i+2}번째 행 : Answer not in options error")
            error_found = True

        # answer가 숫자로 이루어져있는지 확인
        if isinstance(index_2, str) and not index_2.isnumeric():
            print(f"\nQuiz 2 {i+2}번째 행 : Answer in options error")
            error_found = True

        # 3. 정답의 인덱스가 answer_index와 일치하는지 확인
        index_2 = int(index_2)
        if options_2[index_2 - 1] != answer_2:
            print((f"\nQuiz 2 {i+2}번째 행 : 정답의 인덱스가 answer_index와 불일치"))
            #print(f"{options_2}의 정답이 {answer_2}이지만 index {index_2}로 적혀있는 에러")
            error_found = True

        # 4. 문장에 두 대화문이 있는지 확인
        appropriate_blank_english = r'A:|B:'
        english_underscores = re.findall(appropriate_blank_english, quiz_2)
        if len(english_underscores) >= 4:
            print(f"\nQuiz 2 {i+2}번째 행 : 퀴즈에 대화문이 2개 있음")
            error_found = True
    
        # 5. 샘플 스크립트 확인
        sample_script_blank = r'(?!A\b|B\b|A:|B:)[a-zA-Z]+'
        extra_english_texts = re.findall(sample_script_blank, sample_script_kor)

        if extra_english_texts:
            print(f"\n샘플 스크립트 {i+2}번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재")
            print(f"추출된 단어: {extra_english_texts}")
            error_found = True
        
        # 5. 샘플 스크립트 대화가 두 개인지 확인
        appropriate_blank_english = r'A:|B:'
        english_underscores = re.findall(appropriate_blank_english, sample_script_kor)
        if len(english_underscores) >= 5:
            print(f"\n샘플 스크립트 {i+2}번째 행 : 대화가 세 개 이상 존재")
            error_found = True        

    if not error_found:
        print("모든 데이터가 정상입니다다")


In [102]:
preview_quiz_check_valid(df_mid)

100%|██████████| 1110/1110 [00:00<00:00, 21222.17it/s]


샘플 스크립트 2번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Leo', 'Canada', 'Mia', 'Japan']

샘플 스크립트 61번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['one', 'one']

샘플 스크립트 95번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 106번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 111번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['TV', 'TV']

샘플 스크립트 128번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 159번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Sarah', 'Sarah', 'Mark', 'Mark']

샘플 스크립트 161번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['TV', 'TV']

샘플 스크립트 165번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 172번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 200번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 232번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 253번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['C']

샘플 스크립트 253번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 270번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Jamie', 'Riley']

샘플 스크립트 324번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Alex', 'Sarah']

샘플 스크립트 325번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 326번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 336번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 360번째 행 : 대화가 세 개 이상 존재

샘플 스크립트 388번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 

In [97]:
preview_quiz_check_valid(df_elem)

100%|██████████| 769/769 [00:00<00:00, 8793.83it/s]


샘플 스크립트 12번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Alex', 'Alex', 'Maria', 'Maria']

샘플 스크립트 19번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Rose', 'Rose']

샘플 스크립트 26번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Lisa', 'Lisa', 'Lisa']

샘플 스크립트 37번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Linda', 'Linda']

샘플 스크립트 38번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Minnie', 'Minnie']

샘플 스크립트 53번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Emma', 'Emma', 'Mr', 'Johnson', 'Mr', 'Johnson']

샘플 스크립트 63번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['S', 'C', 'H', 'O', 'O', 'L', 'F', 'U', 'N']

샘플 스크립트 67번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['D', 'V', 'I', 'D', 'D', 'V', 'I', 'D']

샘플 스크립트 73번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['T', 'E', 'R', 'M', 'I', 'T', 'E', 'M']

샘플 스크립트 77번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['D', 'V', 'I', 'D', 'CODE', 'CODE', 'C', 'O', 'D', 'E']

샘플 스크립트 86번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Grace', 'Grace', 'Lily']

샘플 스크립트 120번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['Bob', 'Bob', 'Car

In [98]:
preview_quiz_check_valid(df_high)

100%|██████████| 719/719 [00:00<00:00, 16486.65it/s]


샘플 스크립트 25번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['TV']

샘플 스크립트 39번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['AI', 'AI']

샘플 스크립트 216번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['LED']

샘플 스크립트 281번째 행 : A: 또는 B: 이외의 영어 텍스트가 존재
추출된 단어: ['ecosystem', 'ecosystem']


## 중복 정답 여부 확인

In [63]:
from openai import OpenAI
from getpass import getpass
from jinja2 import Template
from openai.lib._parsing import type_to_response_format_param
from pydantic import BaseModel
import json
import pandas as pd
from tqdm import tqdm

In [64]:
api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [108]:
prompt_template = Template("""
[PURPOSE]
You are going to solve the quiz and write down the answer_reason.
There is not necessarily a single correct answer.
Select all that are correct.
                           
[Example]
```
Input: 
    {
    "quiz": "What is your ______ sport?",
    "korean": "네가 가장 좋아하는 스포츠는 무엇이에요?",
    "option_1": "like",
    "option_2": "favorite",
    "option_3": "best",
  }

Output:
    {   
    "possible_answer": ["favorite"],
    "answer_reason": "The word 'favorite' is the correct answer because the sentence is asking about someone's most preferred sport.

1. 'favorite' means 가장 좋아하는 in Korean, which matches the meaning of the given Korean sentence.
2. 'like' is a verb (좋아하다) and does not fit grammatically in this sentence.
3. 'best' means 최고의, which refers to the highest quality rather than personal preference.

Thus, 'favorite' is the correct choice."
}
```                     
```
Input: 
    {
    "quiz": "A: What are you _____ on your holiday? B: I’m going to travel.",
    "korean": "A: 너는 휴일에 무엇을 할 예정이니? B: 여행을 갈 거야.",
    "option_1": "planning to do",
    "option_2": "going to do",
    "option_3": "will do",
  }

Output:
    {   
    "possible_answer": ["planning to do, going to do",]
    "answer_reason": "Both 'planning to do' and 'going to do' fit naturally in the sentence.

1. 'Going to do' is commonly used for future plans and matches the response "I’m going to travel."
2. 'Planning to do' also makes sense because it refers to making arrangements for the holiday.
3. 'Will do' is less natural in this context because it’s not commonly used for asking about planned activities."

Thus, 'planning to do' and 'going to do' are the correct choice."
}
```

[Input]
```
QUIZ : {{quiz}}
KOREAN : {{korean}}
OPTION_1 : {{option_1}}
OPTION_2 : {{option_2}}
OPTION_3 : {{option_3}}
```
""")

In [109]:
class Reason(BaseModel):
    possible_answer : list[str]
    answer_reason : str

In [102]:
response_format = type_to_response_format_param(Reason)

In [103]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'possible_answer': {'items': {'type': 'string'},
     'title': 'Possible Answer',
     'type': 'array'},
    'answer_reason': {'title': 'Answer Reason', 'type': 'string'}},
   'required': ['possible_answer', 'answer_reason'],
   'title': 'Reason',
   'type': 'object',
   'additionalProperties': False},
  'name': 'Reason',
  'strict': True}}

In [69]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You are going to solve the quiz and write down the answer_reason."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = Reason,
    )
    return response.choices[0].message.parsed

In [71]:
prompt = prompt_template.render(
    quiz = "___ I borrow your book?",
    korean = "내 책 좀 빌려도 될까요?",
    option_1 = "May",
    option_2 = "Can",
    option_3 = "Might"
)

In [72]:
response = completion(prompt)

In [74]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent=2)
print(response_output)

{
  "possible_answer": [
    "May",
    "Can"
  ],
  "answer_reason": "The sentence is asking for permission, as indicated by the Korean '내 책 좀 빌려도 될까요?' which translates to 'May I borrow your book?'.\n\n1. 'May' is the traditional modal verb used for asking permission in formal or polite contexts.\n2. 'Can' is commonly used in everyday conversation to ask for permission, even though its primary meaning is about ability.\n3. 'Might' is less commonly used in this context and tends to sound overly formal or archaic, making it less suitable compared to 'May' and 'Can'.\n\nThus, 'May' and 'Can' are both acceptable answers."
}


In [75]:
df_mid.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'title', 'new_DES',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 30', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34'],
      dtype='object')

In [ ]:
def answer_duplication_check_quiz1(data:pd.DataFrame, output_filename = "PreviewQuiz1_MID_duplicate_answer.csv"):
    # 에러 리스트 확인
    duplicate_data = []

    # 파일 변환
    for i in tqdm(range(len(data)), total=len(data)):
        prompt = prompt_template.render(
            quiz = data.loc[i, 'Quiz 1'],
            korean = data.loc[i, 'Korean 1'],
            option_1 = data.loc[i, 'Option 1-1'],
            option_2 = data.loc[i, 'Option 1-2'],
            option_3 = data.loc[i, 'Option 1-3']
        )

        response = client.beta.chat.completions.parse(
            model = "o3-mini",
            reasoning_effort = "low",
            messages = [
                {"role" : "system", "content" : "You are going to solve the quiz and write down the answer_reason."},
                {"role" : "user", "content" : prompt}
            ],
            response_format = Reason
        )

        # 응답 결과 출력 및 결과 출력력
        output = response.choices[0].message.parsed
        possible_answers = output.possible_answer
        answer_reason = output.answer_reason

        # 중복 답변 체크
        if len(possible_answers) > 1:
            row_number = i+2
            duplicate_data.append({"Row": row_number, "Reason": answer_reason})

    # 최종적으로 중복된 문제를 출력
    if duplicate_data:
        df_duplicates = pd.DataFrame(duplicate_data)
        df_duplicates.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n중복된 문제 목록이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("\n중복된 문제가 없습니다.")
    

In [130]:
def answer_duplication_check_quiz2(data:pd.DataFrame,output_filename = "PreviewQuiz2_MID_duplicate_answer.csv"):
    # 에러 리스트 확인
    duplicate_data = []

    # 파일 변환
    for i in tqdm(range(len(data)), total=len(data)):
        prompt = prompt_template.render(
            quiz = data.loc[i, 'Quiz 2'],
            korean = data.loc[i, 'Korean 2'],
            option_1 = data.loc[i, 'Option 2-1'],
            option_2 = data.loc[i, 'Option 2-2'],
            option_3 = data.loc[i, 'Option 2-3']
        )

        response = client.beta.chat.completions.parse(
            model = "o3-mini",
            reasoning_effort = "low",
            messages = [
                {"role" : "system", "content" : "You are going to solve the quiz and write down the answer_reason."},
                {"role" : "user", "content" : prompt}
            ],
            response_format = Reason
        )

        # 응답 결과 출력 및 결과 출력력
        output = response.choices[0].message.parsed
        possible_answers = output.possible_answer
        answer_reason = output.answer_reason

        # 중복 답변 체크
        if len(possible_answers) > 1:
            row_number = i+2
            duplicate_data.append({"Row": row_number, "Reason": answer_reason})

    # 최종적으로 중복된 문제를 출력
    if duplicate_data:
        df_duplicates = pd.DataFrame(duplicate_data)
        df_duplicates.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n중복된 문제 목록이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("\n중복된 문제가 없습니다.")
    

In [128]:
sample = df_mid[:5]

In [129]:
answer_duplication_check_quiz1(sample)

100%|██████████| 5/5 [00:16<00:00,  3.28s/it]


중복된 문제 목록이 'PreviewQuiz_MID_duplicate_answer.csv' 파일에 저장되었습니다.


In [131]:
answer_duplication_check_quiz2(sample)

100%|██████████| 5/5 [00:18<00:00,  3.71s/it]


중복된 문제가 없습니다.


# sample script 검수 코드
- 배치잡으로 돌릴 것

In [26]:
from openai import OpenAI
from getpass import getpass
from jinja2 import Template
from openai.lib._parsing import type_to_response_format_param
from pydantic import BaseModel
import json
import pandas as pd
from tqdm import tqdm

In [37]:
api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [38]:
prompt_template = Template("""
[PERSONA]
You are a translation review expert.

[POLICY]
1. Check if "sample_script_eng" and "sample_script_kor" have the same meaning.
2. Classify the comparison result into one of the following states: "correct", "supplement", or "incorrect".
3. Provide a reason for the assigned state in the "reason" field.
                          
[EXAMPLE 1 : correct]
```
Input:
    {
    "id" : 1,
    "sample_script_eng" : "A: Hello! It's wonderful to see you tonight. B: Good evening! I hope you have a pleasant time. A: Goodbye! Take care and see you soon.",
    "sample_script_kor" : "A: 안녕하세요! 오늘 밤 당신을 보게 되어 반가워요. B: 좋은 저녁이에요! 즐거운 시간 보내길 바래요. A: 잘 가요! 조만간 또 봐요."
    }

Output:
    {
    "id" : 1,
    "state" : "correct",
    "reason": "The Korean translation accurately reflects the meaning of the English script."
    }
```       

[EXAMPLE 2 : supplement]
```
Input:
    {
    "id" : 2,
    "sample_script_eng" : "A: I have a cooking class this week. B: Really? A: I also have an art class on Saturday. B: That sounds exciting!",
    "sample_script_kor" : "A: 나는 이번 주에 수업이 있어. B: 정말? A: 그리고 토요일에 수업도 있어. B: 멋지다!"
    }

Output:
    {
    "id" : 2,
    "state" : "supplement",
    "reason": "The Korean translation mostly preserves the meaning, but "나는 이번 주에 수업이 있어." is too vague compared to "I have a cooking class this week.""
    }
```

[EXAMPLE 3 : incorrect]
```
Input:
    {
    "id" : 3,
    "sample_script_eng" : "A: What will you do on Saturday? B: I will go to the park. A: What are your plans for next week? B: I will start a new hobby.",
    "sample_script_kor" : "A: 너는 몇 시에 일어나니? B: 나는 7시에 일어나. A: 몇 시에 잠자리에 드니? B: 나는 11시에 잠자리에 들어."
    }

Output:
    {
    "id" : 3,
    "state" : "incorrect",
    "reason": "The Korean translation does not match the English script at all. The topics and meanings are completely different."
    }
```

[Input]
```
ID : {{id}}
SAMPLE_SCRIPT_ENG : {{sample_script_eng}}
SAMPLE_SCRIPT_KOR : {{sample_script_kor}}
```
""")

In [39]:
class Reason(BaseModel):
    id : int
    state : str
    reason : str

In [40]:
response_format = type_to_response_format_param(Reason)

In [41]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'id': {'title': 'Id',
     'type': 'integer'},
    'state': {'title': 'State', 'type': 'string'},
    'reason': {'title': 'Reason', 'type': 'string'}},
   'required': ['id', 'state', 'reason'],
   'title': 'Reason',
   'type': 'object',
   'additionalProperties': False},
  'name': 'Reason',
  'strict': True}}

In [42]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You must check in detail whether Korean and English match."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = Reason,
    )
    return response.choices[0].message.parsed

In [43]:
prompt = prompt_template.render(
    id = 35,
    sample_script_eng = "A: How can I get to the museum? B: Take bus number 5 and get off at the final stop. A: Can you confirm the route? B: Absolutely, just follow the route map.",
    sample_script_kor = "A: 어떻게 박물관에 갈 수 있나요? B: 5번 버스를 타고 마지막 정류장에서 내리세요. A: 경로를 다시 확인해 주실래요? B: 물론이죠, 지도에 따라 가시면 됩니다."
)

In [44]:
print(prompt)


[PERSONA]
You are a translation review expert.

[POLICY]
1. Check if "sample_script_eng" and "sample_script_kor" have the same meaning.
2. Classify the comparison result into one of the following states: "correct", "supplement", or "incorrect".
3. Provide a reason for the assigned state in the "reason" field.
                          
[EXAMPLE 1 : correct]
```
Input:
    {
    "id" : 1,
    "sample_script_eng" : "A: Hello! It's wonderful to see you tonight. B: Good evening! I hope you have a pleasant time. A: Goodbye! Take care and see you soon.",
    "sample_script_kor" : "A: 안녕하세요! 오늘 밤 당신을 보게 되어 반가워요. B: 좋은 저녁이에요! 즐거운 시간 보내길 바래요. A: 잘 가요! 조만간 또 봐요."
    }

Output:
    {
    "id" : 1,
    "state" : "correct",
    "reason": "The Korean translation accurately reflects the meaning of the English script."
    }
```       

[EXAMPLE 2 : supplement]
```
Input:
    {
    "id" : 2,
    "sample_script_eng" : "A: I have a cooking class this week. B: Really? A: I also have an art class on Sat

In [45]:
response = completion(prompt)

In [46]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent=2)
print(response_output)

{
  "id": 35,
  "state": "correct",
  "reason": "The Korean translation accurately reflects the meaning of the English script. The dialogue is fully consistent, with the same instructions in terms of directions and inquiries."
}


In [47]:
df_mid.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'title', 'new_DES',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 30', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34'],
      dtype='object')

In [ ]:
# def answer_duplication_check_quiz1(data:pd.DataFrame, output_filename = "PreviewQuiz1_MID_duplicate_answer.csv"):
#     # 에러 리스트 확인
#     duplicate_data = []

#     # 파일 변환
#     for i in tqdm(range(len(data)), total=len(data)):
#         prompt = prompt_template.render(
#             id = data.loc[i,"id"]
#             sample_script_eng = data.loc[i, '샘플 스크립트 (영어)'],
#             sample_script_kor = data.loc[i, '샘플 스크립트 (한국어)']
#         )

#         response = client.beta.chat.completions.parse(
#             model = "o3-mini",
#             reasoning_effort = "low",
#             messages = [
#                 {"role" : "system", "content" : "You must check in detail whether Korean and English match."},
#                 {"role" : "user", "content" : prompt}
#             ],
#             response_format = Reason
#         )

#         # 응답 결과 출력 및 결과 출력력
#         output = response.choices[0].message.parsed
#         possible_answers = output.possible_answer
#         answer_reason = output.answer_reason

#     # 최종적으로 중복된 문제를 출력
#     if duplicate_data:
#         df_duplicates = pd.DataFrame(duplicate_data)
#         df_duplicates.to_csv(output_filename, index=False, encoding='utf-8-sig')
#         print(f"\n중복된 문제 목록이 '{output_filename}' 파일에 저장되었습니다.")
#     else:
#         print("\n중복된 문제가 없습니다.")
    

# 엑셀 전처리

In [205]:
import pandas as pd
db_mid = pd.read_csv("DB_MID.csv")
db_elem = pd.read_csv("DB_Elem.csv")
db_high = pd.read_csv("DB_High.csv")

In [206]:
db_mid.columns

Index(['curriculum id', 'textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표', 'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35'],
      dtype='object')

In [207]:
db_elem.columns

Index(['curriculum id', 'textbook ID', '학년', '출판사', '교육과정', '단원 번호', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'KEY_EXPRESSION',
       'KEY_EXPRESSION_KOR', 'SUB_EXPRESSION', 'SUB_EXPRESSION_KOR', '학습목표',
       'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)'],
      dtype='object')

In [208]:
db_high.columns

Index(['curriculum id', 'textbook ID', '제목', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표', 'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35'],
      dtype='object')

# 퀴즈 1

In [209]:
drop_col_mid = ['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35']
db_mid.drop(columns=drop_col_mid, inplace=True)

In [210]:
drop_col_elem = ['textbook ID', '학년', '출판사', '교육과정', '단원 번호', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'KEY_EXPRESSION',
       'KEY_EXPRESSION_KOR', 'SUB_EXPRESSION', 'SUB_EXPRESSION_KOR', '학습목표',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)']
db_elem.drop(columns=drop_col_elem, inplace=True)

In [211]:
drop_col_high = ['textbook ID', '제목', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35']
db_high.drop(columns=drop_col_high, inplace=True)

In [212]:
db_mid = db_mid.rename(columns={
    'curriculum id' : "id",
    'Quiz 1' : "quiz",
    'Korean 1' : "korean",
    'Option 1-1' : "option_1",
    'Option 1-2' : "option_2",
    'Option 1-3' : "option_3",
})
db_mid.head(3)

,id,quiz,korean,option_1,option_2,option_3
0,1,A: My name is Felix. I’m from ____.,A: 내 이름은 Felix야. 나는 Australia 출신이야. B: 만나서 반가워.,America,Australia,Canada
1,2,What is your ______ sport?,네가 가장 좋아하는 스포츠는 무엇이에요?,like,favorite,best
2,3,A: What are you _____ on your holiday? B: I’m ...,A: 너는 휴일에 무엇을 할 예정이니? B: 여행을 갈 거야.,planning to do,going to do,will do


In [213]:
db_elem = db_elem.rename(columns={
    'curriculum id' : "id",
    'Quiz 1' : "quiz",
    'Korean 1' : "korean",
    'Option 1-1' : "option_1",
    'Option 1-2' : "option_2",
    'Option 1-3' : "option_3",
})
db_elem.head(3)

,id,quiz,korean,option_1,option_2,option_3
0,10001,___!,안녕하세요!,Goodbye!,Hello!,Hi!
1,10002,___ Minho.,나는 민호야.,I do,I’m,I goes
2,10003,A: _____ a lot. B: No problem.,A: 정말 고마워요. B: 별말씀을요.,Thankful,Thanks,Thank


In [214]:
db_high = db_high.rename(columns={
    'curriculum id' : "id",
    'Quiz 1' : "quiz",
    'Korean 1' : "korean",
    'Option 1-1' : "option_1",
    'Option 1-2' : "option_2",
    'Option 1-3' : "option_3",
})
db_high.head(3)

,id,quiz,korean,option_1,option_2,option_3
0,1001,You should ______.,너는 더 많이 자야 해.,eat a snack,get more sleep,take a walk
1,1002,I am fond of ___ new strategies.,저는 새로운 전략을 시도하는 것을 좋아해요.,try,trying,tried
2,1003,She embraces the idea ___ hard work pays off.,그녀는 열심히 일하면 보상이 따른다는 아이디어를 받아들였어요.,if,that,which


## 중등 나누기

In [215]:
len(db_mid)

1110

In [216]:
db_mid_1 = db_mid[:600]
db_mid_2 = db_mid[600:]

print(f"db_mid_1 크기 : {len(db_mid_1)}")
print(f"db_mid_2 크기 : {len(db_mid_2)}")
print(f"총 db 크기 {len(db_mid_1) + len(db_mid_2)}")

# csv 파일 만들기
db_mid_1.to_csv("Quiz1_MID_Duplicate_check_1.csv", encoding="utf-8-sig", index=False)
db_mid_2.to_csv("Quiz1_MID_Duplicate_check_2.csv", encoding="utf-8-sig", index=False)

db_mid_1 크기 : 600
db_mid_2 크기 : 510
총 db 크기 1110


## 초등 나누기

In [217]:
print(len(db_elem))

# csv 파일 만들기
db_elem.to_csv("Quiz1_ELEM_Duplicate_check.csv", encoding="utf-8-sig", index=False)

769


## 고등 나누기

In [218]:
len(db_high)

# csv 파일 만들기
db_high.to_csv("Quiz1_HIGH_Duplicate_check.csv", encoding="utf-8-sig", index=False)

# 퀴즈 2

In [219]:
import pandas as pd
db_mid = pd.read_csv("DB_MID.csv")
db_elem = pd.read_csv("DB_Elem.csv")
db_high = pd.read_csv("DB_High.csv")

In [220]:
db_mid.columns

Index(['curriculum id', 'textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표', 'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35'],
      dtype='object')

In [221]:
db_elem.columns

Index(['curriculum id', 'textbook ID', '학년', '출판사', '교육과정', '단원 번호', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'KEY_EXPRESSION',
       'KEY_EXPRESSION_KOR', 'SUB_EXPRESSION', 'SUB_EXPRESSION_KOR', '학습목표',
       'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)'],
      dtype='object')

In [222]:
db_high.columns

Index(['curriculum id', 'textbook ID', '제목', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표', 'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35'],
      dtype='object')

In [223]:
drop_col_mid = ['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표', 'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35']
db_mid.drop(columns=drop_col_mid, inplace=True)

In [224]:
drop_col_elem = ['textbook ID', '학년', '출판사', '교육과정', '단원 번호', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'KEY_EXPRESSION',
       'KEY_EXPRESSION_KOR', 'SUB_EXPRESSION', 'SUB_EXPRESSION_KOR', '학습목표',
       'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)']
db_elem.drop(columns=drop_col_elem, inplace=True)

In [225]:
drop_col_high = ['textbook ID', '제목', '출판사', '교육과정', 'Lesson', '구분',
       'title', 'new_DES', 'DESCRIPTION', 'Key Expression (영어)',
       'Key Expression (한국어)', 'Sub Expression (영어)', 'Sub Expression (한국어)',
       '학습목표', 'Quiz 1', 'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3',
       'Answer 1', 'Index 1', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 31', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35']
db_high.drop(columns=drop_col_high, inplace=True)

In [226]:
db_mid = db_mid.rename(columns={
    'curriculum id' : "id",
    'Quiz 2' : "quiz",
    'Korean 2' : "korean",
    'Option 2-1' : "option_1",
    'Option 2-2' : "option_2",
    'Option 2-3' : "option_3",
})
db_mid.head(3)

,id,quiz,korean,option_1,option_2,option_3
0,1,"A: Hi, I’m Sam. I ____ from Korea. B: Nice to ...","A: 안녕, 나는 Sam이야. 나는 Korea에서 왔어. B: 만나서 반가워.",live,come,am
1,2,What do you ______ the most?,가장 좋아하는 것은 무엇이에요?,prefer,love,like
2,3,I'm _____ to try a new recipe.,나는 새로운 요리를 시도할 계획이야.,planning,preparing,deciding


In [227]:
db_elem = db_elem.rename(columns={
    'curriculum id' : "id",
    'Quiz 2' : "quiz",
    'Korean 2' : "korean",
    'Option 2-1' : "option_1",
    'Option 2-2' : "option_2",
    'Option 2-3' : "option_3",
})
db_elem.head(3)

,id,quiz,korean,option_1,option_2,option_3
0,10001,Good evening! ___ you soon!,좋은 저녁이에요! 곧 봐요!,See,Look,Meet
1,10002,My name is Minho. I ___ Minho.,내 이름은 민호야. 나는 민호라고 불러.,called,go by,roll by
2,10003,A: I _____ it. B: Anytime.,A: 나는 그것에 감사해. B: 언제든지.,accept,appreciate,ignore


In [228]:
db_high = db_high.rename(columns={
    'curriculum id' : "id",
    'Quiz 2' : "quiz",
    'Korean 2' : "korean",
    'Option 2-1' : "option_1",
    'Option 2-2' : "option_2",
    'Option 2-3' : "option_3",
})
db_high.head(3)

,id,quiz,korean,option_1,option_2,option_3
0,1001,You should ______ every day.,"네, 매일 연습해야 해.",sing,practice,play
1,1002,They are worried about ___ the bus.,그들은 버스를 놓칠까 봐 걱정해요.,miss,missing,missed
2,1003,They maintain the belief ___ every effort counts.,그들은 모든 노력이 중요하다는 믿음을 유지해요.,that,if,whether


## 중등 나누기

In [229]:
len(db_mid)

1110

In [230]:
db_mid_1 = db_mid[:600]
db_mid_2 = db_mid[600:]

print(f"db_mid_1 크기 : {len(db_mid_1)}")
print(f"db_mid_2 크기 : {len(db_mid_2)}")
print(f"총 db 크기 {len(db_mid_1) + len(db_mid_2)}")

# csv 파일 만들기
db_mid_1.to_csv("Quiz2_MID_Duplicate_check_1.csv", encoding="utf-8-sig", index=False)
db_mid_2.to_csv("Quiz2_MID_Duplicate_check_2.csv", encoding="utf-8-sig", index=False)

db_mid_1 크기 : 600
db_mid_2 크기 : 510
총 db 크기 1110


## 초등 나누기기

In [231]:
len(db_elem)

769

In [232]:
print(len(db_elem))

# csv 파일 만들기
db_elem.to_csv("Quiz2_ELEM_Duplicate_check.csv", encoding="utf-8-sig", index=False)

769


## 고등 나누기

In [233]:
len(db_high)

# csv 파일 만들기
db_high.to_csv("Quiz2_HIGH_Duplicate_check.csv", encoding="utf-8-sig", index=False)